<a href="https://colab.research.google.com/github/Djadih/DataScience_Project/blob/main/midi_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import string
import numpy as np
from music21 import converter, instrument, note, chord, stream, tempo, meter
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [2]:
 from google.colab import files
uploaded = files.upload()

Saving beethoven_opus10_1.mid to beethoven_opus10_1.mid
Saving beethoven_opus10_2.mid to beethoven_opus10_2.mid
Saving beethoven_opus10_3.mid to beethoven_opus10_3.mid
Saving pathetique_1.mid to pathetique_1.mid
Saving pathetique_2.mid to pathetique_2.mid
Saving pathetique_3.mid to pathetique_3.mid


In [18]:
def get_notes(filename):
  buffer_size = 500000
  notes = [''] * buffer_size
  last_time = 0
  offset = []
  midi = converter.parse(filename)
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    if isinstance(element, note.Note):
      time = int(element.offset * 4)
      if notes[time] == '':
        notes[time] = str(element.pitch.midi)
      last_time = time
    elif isinstance(element, chord.Chord):
      time = int(element.offset * 4)
      if notes[time] == '':
        notes[time] = '.'.join(str(n) for n in element.normalOrder) 
      last_time = time
  print(last_time)
  return notes[:last_time]

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 400

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))
    
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    i  = []
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, normalized_input, network_output)

def normalize(network_input, n_vocab):
    sequence_length = 400
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    return normalized_input

In [12]:
notes = get_notes('beethoven_opus10_1.mid')

4644


In [5]:
len(notes)

4644

In [10]:
n_vocab = len(set(notes))
n_vocab

101

In [13]:
network_input, normalized_input,  network_output = prepare_sequences(notes, n_vocab)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(network_input, network_output, test_size=0.33, random_state=42)

In [19]:
X_train_normalized = normalize(X_train, n_vocab)

In [24]:
X_train_normalized.shape

(2843, 400, 1)

In [25]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [26]:
model = create_network(normalized_input, n_vocab)

In [ ]:
model.fit(X_train_normalized, y_train, epochs=50, batch_size=128)

Epoch 1/50
23/23 [==============================] - 6s 247ms/step - loss: 0.6058
Epoch 2/50
23/23 [==============================] - 6s 250ms/step - loss: 0.5938
Epoch 3/50
23/23 [==============================] - 6s 253ms/step - loss: 0.5882
Epoch 4/50
23/23 [==============================] - 6s 256ms/step - loss: 0.5778
Epoch 5/50
23/23 [==============================] - 6s 253ms/step - loss: 0.5821
Epoch 6/50
23/23 [==============================] - 6s 250ms/step - loss: 0.5637
Epoch 7/50
23/23 [==============================] - 6s 247ms/step - loss: 0.5531
Epoch 8/50
23/23 [==============================] - 6s 246ms/step - loss: 0.5444
Epoch 9/50
23/23 [==============================] - 6s 246ms/step - loss: 0.5371
Epoch 10/50
23/23 [==============================] - 6s 244ms/step - loss: 0.5358
Epoch 11/50
23/23 [==============================] - 6s 245ms/step - loss: 0.5207
Epoch 12/50
17/23 [=====================>........] - ETA: 1s - loss: 0.5107

In [38]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    end = len(network_input)
    start = np.random.randint(0, end)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [39]:
pitchnames = sorted(set(item for item in notes))
output = generate_notes(model, X_test, pitchnames, n_vocab)

In [40]:
output[:30]

['',
 '54',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48',
 '',
 '53',
 '',
 '48']

In [41]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """

    output_notes = []

    # create note and chord objects based on the values generated by the model
    for idx, pattern in enumerate(prediction_output):
      if len(pattern) != 0:
        notes_in_chord = [int(n) for n in pattern.split('.')]
        new_chord = chord.Chord(notes_in_chord)
        new_chord.offset = idx/4
        output_notes.append(new_chord)
        # increase offset each iteration so that notes do not stack

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [42]:
create_midi(output)